In [1]:
import numpy as np
import pandas as pd

#import data
df = pd.read_csv('/Users/dominicwelti/Library/CloudStorage/Dropbox/Master_Thesis/data_npj/Graph_data.csv')

In [2]:
def calculate_rel_e(df:pd.DataFrame):
    '''Calculate normalized error. Returns dataframe with new column ["Normalized error"].'''
    df['Normalized error']=0
    for (i, error_e, error_f) in zip(df.index, df['Error_E'],df['Error_F']):
        df.loc[i,'Normalized error']=(1/2)*( error_f/max(df['Error_F']) + error_e/max(df['Error_E']) )
    return df

In [3]:
df=calculate_rel_e(df)

In [43]:
import plotly.express as px

fig = px.scatter(df, x='Cost', y='Normalized error', color='Type', text='Model', log_x=True)
fig.update_layout(
    xaxis_title=dict(text='Computational cost of inference [<i>μs/atom/step</i>]')
) 
#fig.update_layout(legend=dict(
#    yanchor
#))

In [148]:
import plotly.graph_objects as go

layout = go.Layout(
    yaxis1=dict(domain=[0.,0.7]),
    yaxis2=dict(domain=[0.75,1])
)

# add assignment to y subplot
df['y axes']='y1'
df.loc[(df['Normalized error']>0.2),'y axes']='y2'

fig = go.Figure(data=[], layout=layout)

for kind in df['Type'].unique():
    subset=df.query('Type==@kind')
    fig.add_trace(go.Scatter(
        x=subset['Cost'],
        y=subset['Normalized error'],
        mode='markers',
        name=kind,
        text=subset['Model']+'<br>'+'Engine: '+subset['MD Suite']+'-'+subset['Engine'],
        hovertemplate="%{text}"+"<br>Computational cost: %{x} μs"+"<br>Normalized error: %{y}",
        legendgroup=kind,
        yaxis=subset['y axes'].unique()[0],
    ),
    )
fig.update_xaxes(title_text='Computational cost of inference [<i>μs/atom/step</i>]',type="log",minor_ticks='inside')
fig.update_layout(yaxis1={'title':'Normalized error','minor_ticks':'inside'},yaxis2={'minor_ticks':'inside'},grid_yaxes=['y1']) #,'position':0

In [157]:
df['Colour']=''
colours=['blue','red','green','purple','orange', 'turquoise']

for i, name in enumerate(df['Type'].unique()):
    df.loc[(df['Type']==name),['Colour']]=colours[i]

In [159]:
import plotly.graph_objects as go
#from colorhash import ColorHash


fig = go.Figure(data=[])

i=0 # index for trace
traces=[] 
for engine in df['Engine'].unique():
    for suite in df['MD Suite'].unique():
        for kind in df['Type'].unique():
            subset=df.query('Type==@kind & Engine==@engine & `MD Suite`==@suite')
            if len(subset)>=1:
                i+=1
                fig.add_trace(go.Scatter(
                    x=subset['Cost'],
                    y=subset['Normalized error'],
                    mode='markers',
                    name=kind,
                    text=subset['Model']+'<br>'+'Engine: '+subset['MD Suite']+'-'+subset['Engine'],
                    hovertemplate="%{text}"+"<br>Computational cost: %{x} μs"+"<br>Normalized error: %{y}",
                    legendgroup=kind,
                    marker={'color':subset['Colour']},
                    #legendgrouptitle=kind,
                    #yaxis=subset['y axes'].unique()[0],
                ))
                trace={
                    'Trace': i,
                    'Type': kind,
                    'Engine': engine,
                    'MD Suite': suite,
                }
                traces.append(trace)

traces=pd.DataFrame(traces) # convert list of dictionaries to DataFrame

# Add annotation
fig.update_xaxes(title_text='Computational cost of inference [<i>μs/atom/step</i>]',type="log",minor_ticks='inside')
fig.update_layout(yaxis={'title':'Normalized error','minor_ticks':'inside'}) #,'position':0

names=set()
fig.for_each_trace(
    lambda trace:
        trace.update(showlegend=False)
        if (trace.name in names) else names.add(trace.name))



In [160]:
def produce_buttons(df:pd.DataFrame, traces:pd.DataFrame):
    '''Produces buttons list for drop down menu'''
    vis_eng=[]
    for engine_sub in df['Engine'].unique():
        entries=[]
        for i, engine in enumerate(traces['Engine']):
            if engine_sub==engine:
                entry=True
            else:
                entry=False
            entries.append(entry)
        vis_eng.append(entries)

    vis_md=[]
    for md_sub in df['MD Suite'].unique():
        entries=[]
        for i, md in enumerate(traces['MD Suite']):
            if md_sub==md:
                entry=True
            else:
                entry=False
            entries.append(entry)
        vis_md.append(entries)

    lst_eng=[]
    lst_eng.append({
            'label': 'All',
            'args': [
                        {'visible': len(vis_eng[0])*[True]}
                        ],
            'method':'update'
        })
    for i, engine in enumerate(df['Engine'].unique()):
        lst_eng.append({
            'label': engine,
            'args': [
                        {'visible': vis_eng[i]}
                        ],
            'method':'update'
        })
    
    lst_md=[]
    lst_md.append({
            'label': 'All',
            'args': [
                        {'visible': len(vis_md[0])*[True]}
                        ],
            'method':'update'
        })
    for i, md in enumerate(df['MD Suite'].unique()):
        lst_md.append({
            'label': md,
            'args': [
                        {'visible': vis_md[i]}
                        ],
            'method':'update'
        })

    return lst_eng, lst_md

In [161]:
# Add dropdown

lst_eng, lst_md = produce_buttons(df, traces)
fig.update_layout(
    updatemenus=[
        dict(
            buttons=lst_eng,
            direction="down",
            x=0.05,
            y=1.05
            #type = "buttons",
            #active=0
        ),
        dict(
            buttons=lst_md,
            direction="down",
            x=0.15,
            y=1.05
            #type = "buttons",
            #active=0
        )
    ]
)

# fig.update_layout(
#     annotations=[
#         dict(text="Engine", x=0, xref="paper", y=1.1, yref="paper",
#                              align="left", showarrow=False),
#         dict(text="MD Suite", x=0.075, xref="paper", y=1.1,
#                              yref="paper", showarrow=False)])

In [145]:
fig.write_html('/Users/dominicwelti/Library/CloudStorage/Dropbox/Master_Thesis/data_npj/Inference.html')